### OVR & OVO 모듈 활용

(1) 모듈 로딩 & 데이터 준비

In [1]:
## 모듈 로딩
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np

In [2]:
# 데이터 준비
data_file = '../Data/fish.csv'

fishDF = pd.read_csv(data_file)
fishDF.head(2)

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.52,4.0200
1,Bream,290.0,26.3,31.2,12.48,4.3056


(2) 데이터셋 준비 <hr>

(2-1) 피쳐 / 타겟 분리

In [3]:
featureDF = fishDF[fishDF.columns[1:]]
targetSR = fishDF[fishDF.columns[0]]

featureDF.shape, targetSR.shape

((159, 5), (159,))

(2-2) 학습용, 테스트용 데이터셋 분리

In [4]:
from sklearn.model_selection import train_test_split

In [12]:
x_train, x_test, y_train, y_test = train_test_split(featureDF, targetSR, stratify=targetSR, test_size=.2, random_state=11)

In [13]:
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((127, 5), (127,), (32, 5), (32,))

(3) 학습 진행 <hr>

In [14]:
## OvO / OvR 에서 사용할 관측치 (Estimator) 생성
model = LogisticRegression(solver='liblinear')

(3-1) OvO 기반 학습 진행

In [15]:
ovoModel = OneVsOneClassifier(model)
ovoModel.fit(x_train, y_train)

OneVsOneClassifier(estimator=LogisticRegression(solver='liblinear'))

In [16]:
# 모델 파라미터 확인
print(f'classes_ : {ovoModel.classes_}')
print(f'feature_names_in : {ovoModel.feature_names_in_}')
print(f'estimators_ : {len(ovoModel.estimators_)}개')        # 7C2 = 21

classes_ : ['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']
feature_names_in : ['Weight' 'Length' 'Diagonal' 'Height' 'Width']
estimators_ : 21개


In [17]:
# 평가
print(f'[Train Score] {ovoModel.score(x_train, y_train)}\n[Test Score] {ovoModel.score(x_test, y_test)}')

[Train Score] 0.9606299212598425
[Test Score] 0.9375


In [18]:
# 예측
ovoModel.predict(x_test[:2])    # 2개 행 예측

array(['Perch', 'Perch'], dtype=object)

In [20]:
# 각 분류기의 선형식에서 계산한 값
ovoModel.decision_function(x_test[:2])

array([[ 0.69414102,  2.3246776 ,  6.33057526,  3.30771052,  5.31771535,
        -0.33155473,  4.29556622],
       [-0.31792142,  4.31397816,  6.32711513,  1.70928551,  5.31720917,
         1.67271767,  1.70852143]])

(3-2) OvR 기반 학습 진행

In [26]:
ovrModel = OneVsRestClassifier(model)
ovrModel.fit(x_train, y_train)

OneVsRestClassifier(estimator=LogisticRegression(solver='liblinear'))

In [27]:
# 평가
print(f'[Train Score] {ovrModel.score(x_train, y_train)}\n[Test Score] {ovrModel.score(x_test, y_test)}')

[Train Score] 0.952755905511811
[Test Score] 0.96875
